In [1]:
import numpy as np
import inspect

# -----------------------------------------------------------
# 1. Build the data dictionary
# -----------------------------------------------------------
def make_data_dict(xy, xrange=None):
    """
    Create a standardized data dictionary for LmfitGlobal.

    Args:
        xy (array-like): 2D array, shape (N, 1 + n_datasets).
                        Column 0 = x
                        Columns 1..n = y datasets.
        xrange (tuple or None): (xmin, xmax) to restrict fitting range.

    Returns:
        dict: valid data dictionary.
    """
    xy = np.asarray(xy)
    if xy.ndim != 2 or xy.shape[1] < 2:
        raise ValueError("`xy` must be a 2D array with columns [x, y1, y2, ...].")

    if xrange is not None:
        if not (isinstance(xrange, (tuple, list)) and len(xrange) == 2):
            raise ValueError("`xrange` must be tuple/list: (xmin, xmax).")

    return {
        "xy": xy,
        "xrange": xrange,
    }


# -----------------------------------------------------------
# 2. Build the function dictionary
# -----------------------------------------------------------
def make_function_dict(func_list, connectors=None):
    """
    Build the function specification dictionary for LmfitGlobal.

    Args:
        func_list (list of dict):
            Each dict must contain:
                - 'func_name': callable
                - 'init_params': dict of parameter hints
                - 'func_kws': dict of extra keyword arguments

        connectors (list of str or None):
            Operators connecting the models: ['+', '-', '*', '/'].
            Must have length == len(func_list) - 1.
            If None and only one function, auto-set to [].

    Returns:
        dict: function dictionary ready for LmfitGlobal
    """
    # ---- validation ----
    if not isinstance(func_list, list) or len(func_list) == 0:
        raise ValueError("func_list must be a non-empty list of model specs.")

    for entry in func_list:
        if "func_name" not in entry:
            raise ValueError("Each function spec must include 'func_name'")
        if not callable(entry["func_name"]):
            raise TypeError("'func_name' must be a Python callable (a function).")

        if "init_params" not in entry or not isinstance(entry["init_params"], dict):
            raise ValueError("'init_params' must be a dict of parameter specifications.")

        if "func_kws" not in entry:
            entry["func_kws"] = {}
        elif not isinstance(entry["func_kws"], dict):
            raise ValueError("'func_kws' must be a dict.")

        # Signature safety check:
        _check_function_signature(entry["func_name"], entry["init_params"])

    # ---- connectors ----
    if connectors is None:
        connectors = [] if len(func_list) == 1 else None

    if connectors is None:
        raise ValueError(
            "connectors must be provided for multi-function models.\n"
            f"Expected list of length {len(func_list)-1}."
        )

    if not isinstance(connectors, list):
        raise TypeError("connectors must be a list of operator strings.")

    if len(connectors) != len(func_list) - 1:
        raise ValueError(
            f"Number of connectors ({len(connectors)}) must be n_functions - 1 "
            f"({len(func_list)-1})."
        )

    allowed_ops = {"+", "-", "*", "/"}
    for op in connectors:
        if op not in allowed_ops:
            raise ValueError(f"Operator '{op}' not allowed. Use {allowed_ops}.")

    return {
        "theory": func_list,
        "theory_connectors": connectors,
    }


# -----------------------------------------------------------
# 3. Signature checking helper
# -----------------------------------------------------------
def _check_function_signature(func, init_params):
    """
    Ensure the function signature matches the provided parameter names.
    Does NOT enforce strict equality but warns if names mismatch.
    """
    sig = inspect.signature(func)
    allowed = list(sig.parameters.keys())[1:]  # first parameter is x
    provided = list(init_params.keys())

    unknown = [p for p in provided if p not in allowed]
    missing = [p for p in allowed if p not in provided]

    if unknown:
        print(f"[WARNING] Function {func.__name__}: unknown init_params {unknown} "
              f"(not in function signature: {allowed})")

    if missing:
        print(f"[INFO] Function {func.__name__}: missing parameters {missing} — "
              "using function defaults or may be required at evaluation.")


# -----------------------------------------------------------
# 4. Build the full items dictionary for LmfitGlobal
# -----------------------------------------------------------
def make_items(data_dict, function_dict):
    """
    Combine data and function dictionaries into one unified `items` dict.

    Args:
        data_dict (dict): from make_data_dict
        function_dict (dict): from make_function_dict

    Returns:
        dict: items suitable for passing to LmfitGlobal
    """
    if not isinstance(data_dict, dict):
        raise TypeError("data_dict must be a dict from make_data_dict")

    if not isinstance(function_dict, dict):
        raise TypeError("function_dict must be a dict from make_function_dict")

    return {
        "data": data_dict,
        "functions": function_dict,
    }


In [ ]:
# data_dict = make_data_dict(
#     xy=xy_dat,
#     xrange=None
# )

# func_list = [
#     {
#         'func_name': step,
#         'init_params': {
#             'amplitude': {'value':100, 'vary':True},
#             'center': {'value':2.5, 'min':0, 'max':10},
#             'sigma': {'value':1},
#         },
#         'func_kws': {'form': 'erf'}
#     },
#     {
#         'func_name': linear,
#         'init_params' : {
#             'slope': {'value':0.0, 'vary':True},
#             'intercept': {'value':0},
#         },
#         'func_kws': {}
#     }
# ]

# function_dict = make_function_dict(
#     func_list,
#     connectors=['+']
# )

# items = make_items(data_dict, function_dict)

# lg = LmfitGlobal(items)
# result = lg.fit()


In [3]:
import numpy as np
import inspect

# -----------------------------------------------------------
# 1. Build the data dictionary
# -----------------------------------------------------------
def make_data_dict(xy, xrange=None):
    xy = np.asarray(xy)
    if xy.ndim != 2 or xy.shape[1] < 2:
        raise ValueError("`xy` must be a 2D array with columns [x, y1, y2, ...].")
    if xrange is not None:
        if not (isinstance(xrange, (tuple, list)) and len(xrange) == 2):
            raise ValueError("`xrange` must be tuple/list: (xmin, xmax).")
    return {"xy": xy, "xrange": xrange}


# -----------------------------------------------------------
# 2. Build the function dictionary
# -----------------------------------------------------------
def make_function_dict(func_list, connectors=None):
    if not isinstance(func_list, list) or len(func_list) == 0:
        raise ValueError("func_list must be a non-empty list of model specifications.")

    for entry in func_list:
        if "func_name" not in entry:
            raise ValueError("Each function spec must include 'func_name'")
        if not callable(entry["func_name"]):
            raise TypeError("'func_name' must be callable (a Python function).")
        if "init_params" not in entry or not isinstance(entry["init_params"], dict):
            raise ValueError("'init_params' must be a dict.")
        if "func_kws" not in entry:
            entry["func_kws"] = {}
        _check_function_signature(entry["func_name"], entry["init_params"])

    if connectors is None:
        connectors = [] if len(func_list) == 1 else None

    if connectors is None:
        raise ValueError(f"connectors must be provided for {len(func_list)} models.")

    if len(connectors) != len(func_list) - 1:
        raise ValueError("Number of connectors must be (n_functions - 1).")

    allowed = {"+", "-", "*", "/"}
    for op in connectors:
        if op not in allowed:
            raise ValueError(f"Unsupported operator '{op}'. Allowed: {allowed}.")

    return {"theory": func_list, "theory_connectors": connectors}


# -----------------------------------------------------------
# 3. Signature validation helper
# -----------------------------------------------------------
def _check_function_signature(func, init_params):
    sig = inspect.signature(func)
    argnames = list(sig.parameters.keys())[1:]  # skip x
    provided = list(init_params.keys())
    unknown = [p for p in provided if p not in argnames]
    missing = [p for p in argnames if p not in provided]
    if unknown:
        print(f"[WARNING] {func.__name__}: unknown init parameters {unknown}. Function args={argnames}")
    if missing:
        print(f"[INFO] {func.__name__}: missing init parameters {missing}.")


# -----------------------------------------------------------
# 4. Make items
# -----------------------------------------------------------
def make_items(data_dict, function_dict):
    return {"data": data_dict, "functions": function_dict}


In [4]:
def make_items_from_xy(x, y_list, func_list, connectors=None, xrange=None):
    """
    Build full items dict from simple inputs.

    Args:
        x (array-like): 1D x array.
        y_list (list of arrays): each y is same length as x.
        func_list: list of function specs (see examples)
        connectors: operators connecting functions
        xrange: optional tuple (xmin, xmax)

    Returns:
        items dict
    """
    x = np.asarray(x)
    y_list = [np.asarray(y) for y in y_list]
    for y in y_list:
        if len(y) != len(x):
            raise ValueError("All y arrays must have same length as x.")

    xy = np.column_stack([x] + y_list)
    data_dict = make_data_dict(xy, xrange)
    function_dict = make_function_dict(func_list, connectors)
    return make_items(data_dict, function_dict)


In [5]:
def make_items_from_dict(x, y_dict, func_list, connectors=None, xrange=None):
    """
    Build items when y datasets are provided in a dictionary.

    Example:
        y_dict = {
            "sample1": y1,
            "sample2": y2,
            "sample3": y3,
        }
    """
    x = np.asarray(x)
    names = list(y_dict.keys())
    y_list = [np.asarray(y_dict[k]) for k in names]

    return make_items_from_xy(x, y_list, func_list, connectors, xrange)


✅ PART 3 — HIGH-LEVEL GlobalFitBuilder CLASS

A fluent, chainable interface.

Usage example:

builder = (
    GlobalFitBuilder()
    .set_data(x, [y0, y1])
    .add_model(step, init_params_step, func_kws={"form": "erf"})
    .add_model(linear, init_params_lin)
    .connect("+")
)

items = builder.build()
lg = LmfitGlobal(items)
result = lg.fit()


In [11]:
class GlobalFitBuilder:
    """
    Fluent interface for building LmfitGlobal items.
    """

    def __init__(self):
        self._x = None
        self._y_list = None
        self._func_list = []
        self._connectors = []
        self._xrange = None

    # --------------------- DATA -------------------------
    def set_data(self, x, y_list, xrange=None):
        self._x = np.asarray(x)
        self._y_list = [np.asarray(y) for y in y_list]
        self._xrange = xrange
        return self

    # --------------------- MODELS ------------------------
    def add_model(self, func, init_params, func_kws=None):
        func_kws = func_kws or {}
        self._func_list.append({
            "func_name": func,
            "init_params": init_params,
            "func_kws": func_kws
        })
        return self

    # ------------------ CONNECTORS -----------------------
    def connect(self, *ops):
        """
        Provide operators connecting added models.
        Example:
            .connect("+", "*")
        """
        self._connectors = list(ops)
        return self

    # ----------------------- BUILD -----------------------
    def build(self):
        if self._x is None or self._y_list is None:
            raise ValueError("Call set_data(x, y_list) before build().")

        items = make_items_from_xy(
            self._x,
            self._y_list,
            func_list=self._func_list,
            connectors=self._connectors,
            xrange=self._xrange
        )
        return items


In [13]:
import os
import numpy as np
from scipy.special import erf, erfc
log2 = np.log(2)
s2pi = np.sqrt(2*np.pi)
s2 = np.sqrt(2.0)
# tiny had been numpy.finfo(numpy.float64).eps ~=2.2e16.
# here, we explicitly set it to 1.e-15 == numpy.finfo(numpy.float64).resolution
tiny = 1.0e-15

def not_zero(value):
    """Return value with a minimal absolute size of tiny, preserving the sign.

    This is a helper function to prevent ZeroDivisionError's.

    Parameters
    ----------
    value : scalar
        Value to be ensured not to be zero.

    Returns
    -------
    scalar
        Value ensured not to be zero.

    """
    return float(np.copysign(max(tiny, abs(value)), value))

def step(x, amplitude=1.0, center=0.0, sigma=1.0, form='linear'):
    """Return a step function.

    Starts at 0.0, ends at `sign(sigma)*amplitude`, has a half-max at
    `center`, rising or falling with `form`:

    - `'linear'` (default) = amplitude * min(1, max(0, arg + 0.5))
    - `'atan'`, `'arctan'` = amplitude * (0.5 + atan(arg)/pi)
    - `'erf'`              = amplitude * (1 + erf(arg))/2.0
    - `'logistic'`         = amplitude * [1 - 1/(1 + exp(arg))]

    where ``arg = (x - center)/sigma``.

    Note that ``sigma > 0`` gives a rising step, while ``sigma < 0`` gives
    a falling step.
    """
    out = np.sign(sigma)*(x - center)/max(tiny*tiny, abs(sigma))

    if form == 'erf':
        out = 0.5*(1 + erf(out))
    elif form == 'logistic':
        out = 1. - 1./(1. + np.exp(out))
    elif form in ('atan', 'arctan'):
        out = 0.5 + np.arctan(out)/np.pi
    elif form == 'linear':
        out = np.minimum(1, np.maximum(0, out + 0.5))
    else:
        msg = (f"Invalid value ('{form}') for argument 'form'; should be one "
               "of 'erf', 'logistic', 'atan', 'arctan', or 'linear'.")
        raise ValueError(msg)

    return amplitude*out


def linear(x, slope=1.0, intercept=0.0):
    """Return a linear function.

    linear(x, slope, intercept) = slope * x + intercept

    """
    return slope * x + intercept


x = np.linspace(0, 10, 200)
y0 = step(x, amplitude=100, center=3, sigma=1, form='erf') + linear(x, 1.0, 0.0)
y1 = step(x, amplitude=80, center=4, sigma=1.2, form='erf') + linear(x, 0.5, 1.0)

y_list = [y0, y1]

init_step = {
    'amplitude': {'value':100, 'vary':True},
    'center': {'value':2.5, 'min':0, 'max':10},
    'sigma': {'value':1}
}

init_linear = {
    'slope': {'value':0.0, 'vary':True},
    'intercept': {'value':0}
}


builder = (
    GlobalFitBuilder()
    .set_data(x, y_list)                      # x and all y datasets
    .add_model(step, init_step, func_kws={"form": "erf"})
    .add_model(linear, init_linear)
    .connect("+")                             # how to combine the 2 functions
)

items = builder.build()

items

[INFO] step: missing init parameters ['form'].


{'data': {'xy': array([[0.00000000e+00, 1.10452485e-03, 1.00009714e+00],
         [5.02512563e-02, 5.17637657e-02, 1.02525533e+00],
         [1.00502513e-01, 1.02563728e-01, 1.05042386e+00],
         [1.50753769e-01, 1.53549231e-01, 1.07560580e+00],
         [2.01005025e-01, 2.04778070e-01, 1.10080508e+00],
         [2.51256281e-01, 2.56324331e-01, 1.12602673e+00],
         [3.01507538e-01, 3.08282446e-01, 1.15127708e+00],
         [3.51758794e-01, 3.60772067e-01, 1.17656415e+00],
         [4.02010050e-01, 4.13943923e-01, 1.20189802e+00],
         [4.52261307e-01, 4.67986762e-01, 1.22729132e+00],
         [5.02512563e-01, 5.23135543e-01, 1.25275979e+00],
         [5.52763819e-01, 5.79680997e-01, 1.27832302e+00],
         [6.03015075e-01, 6.37980694e-01, 1.30400524e+00],
         [6.53266332e-01, 6.98471728e-01, 1.32983633e+00],
         [7.03517588e-01, 7.61685134e-01, 1.35585297e+00],
         [7.53768844e-01, 8.28262060e-01, 1.38210004e+00],
         [8.04020101e-01, 8.98971759e-01, 

✅ PART 4 — YAML / JSON SCHEMA SUPPORT

This allows external configuration, e.g.:

YAML example

data:
  xy: "data.csv"
  xrange: [0, 10]

functions:
  theory:
    - func_name: step
      init_params:
        amplitude: {value: 100, vary: true}
        center: {value: 2.5, min: 0, max: 10}
        sigma: {value: 1}
      func_kws: {form: erf}

    - func_name: linear
      init_params:
        slope: {value: 0.0}
        intercept: {value: 0}
      func_kws: {}

  theory_connectors: ["+"]


In [ ]:
import yaml
import importlib

def load_items_from_yaml(path):
    with open(path, "r") as f:
        cfg = yaml.safe_load(f)

    data_cfg = cfg["data"]
    func_cfg = cfg["functions"]

    # Load xy: support filenames or arrays
    xy = data_cfg["xy"]
    if isinstance(xy, str):
        xy = np.loadtxt(xy, delimiter=",")
    xrange = data_cfg.get("xrange")

    data_dict = make_data_dict(xy, xrange)

    # turn function names into callables
    func_list = []
    for spec in func_cfg["theory"]:
        func_name = spec["func_name"]
        if isinstance(func_name, str):
            module = importlib.import_module("__main__")   # OR your funcs module
            func = getattr(module, func_name)
        else:
            func = func_name
        func_list.append({
            "func_name": func,
            "init_params": spec["init_params"],
            "func_kws": spec.get("func_kws", {})
        })

    connectors = func_cfg.get("theory_connectors", None)
    function_dict = make_function_dict(func_list, connectors)

    return make_items(data_dict, function_dict)
